In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy

In [2]:
data=pd.read_csv("Train_Data.csv")
data.head()

,text,author,controversiality,parent_text,parent_score,parent_votes,parent_author,parent_controversiality,Score
0,i must be retarded i thought it meant con lawl...,['calantus'],0,"It's quite unfair to call Hillary Clinton a ""c...",245,245,Whisper,0,-8
1,DOWNMODDED FOR IRRELEVANCE? ISN'T THAT HOW THI...,['Shadowrose'],0,upmodded for awesome kindness,32,32,b3mus3d,0,-16
2,"THAT WAS SUPPOSED TO MEAN "" BY A PLACE WHERE P...",['NExusRush'],0,"What the hell does ""because its by a golf cour...",12,12,mr_jellyneck,0,-7
3,I THOUGHT EVERYONE DID; ITS FUCKING DELICIOUS :\,['R0N_SWANS0N'],0,NICE TRY JENNIFER! I KNOW IT'S YOU AND I KNOW...,117,117,ometzo,0,67
4,"Great work, Zhesbe! I'd give you a raise but y...",['reddums'],0,"""HEY BOSS COME LOOK AT WHAT I DID!""",1933,1933,Zhesbe,0,1348


In [3]:
data.controversiality.value_counts()

0    4996
1       3
Name: controversiality, dtype: int64

In [94]:
df1=data.drop(["author","controversiality","parent_text","parent_votes","parent_author","parent_controversiality"],axis=1)

In [95]:
df1

,text,parent_score,Score
0,i must be retarded i thought it meant con lawl...,245,-8
1,DOWNMODDED FOR IRRELEVANCE? ISN'T THAT HOW THI...,32,-16
2,"THAT WAS SUPPOSED TO MEAN "" BY A PLACE WHERE P...",12,-7
3,I THOUGHT EVERYONE DID; ITS FUCKING DELICIOUS :\,117,67
4,"Great work, Zhesbe! I'd give you a raise but y...",1933,1348
...,...,...,...
4994,"Dying words of my father: ""Son, one day a man ...",540,234
4995,CATERING TO EVERYONE AND THEIR IMPOSSIBLE SIMU...,560,107
4996,RABBLERABBLERABBLERABBLE!,370,193
4997,"LITTLE KNOWN FACT, ""VIOLA"" IS NOT ONLY A FRENC...",4,-8


In [43]:
from nltk import RegexpTokenizer
import nltk
from spacytextblob.spacytextblob import SpacyTextBlob
nlp=spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')
regex=r'[A-Za-z]+'
tokenizer=RegexpTokenizer(regex,gaps=False)
words = set(nltk.corpus.words.words())

In [63]:
def text_preprocess(text):
    text=" ".join([i.lower() for i in text.split(" ")])
    doc=nlp(text)
    t1=[i.lemma_ for i in doc if not(i.is_space or i.is_punct) if not i.is_stop]
    t1=(token for token in tokenizer.tokenize(" ".join(t1)) if token.lower() in words or not token.isalpha() )
    return(" ".join(t1))

In [64]:
df1["preprocessed_text"]=df1.text.apply(text_preprocess)

In [65]:
df2=df1.drop("text",axis=1)

In [102]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()

In [74]:
X=pd.DataFrame(vectorizer.fit_transform(df2.preprocessed_text).toarray(),columns=vectorizer.get_feature_names_out())
Y=df2.Score

In [70]:
X

,aa,abandon,abbreviation,abduct,abide,ability,able,abnormal,aboard,abolish,...,young,youngster,youth,youthful,yr,zeppelin,zero,zip,zombie,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
from sklearn.model_selection import train_test_split
x_train,x_text,y_train,y_test=train_test_split(X,Y,test_size=.2,random_state=42)

In [104]:
from sklearn.linear_model import LinearRegression
reg=LinearRegression(n_jobs=5)

In [105]:
reg.fit(x_train,y_train)

LinearRegression(n_jobs=5)